# Tiltmeter - Calibration - Test Run 4

### Libraries

In [1]:
from andbro__querrySeismoData import __querrySeismoData
from andbro__readYaml import __readYaml
from andbro__get_seconds import __get_seconds
from andbro__calculate_polynomial_fit import __calculate_polynomial_fit
from andbro__calculate_linear_regression import __calculate_linear_regression

from scipy import signal
from obspy import UTCDateTime, read, Stream, Trace


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

### Methods

In [2]:
def __removeLinearTrend(data):

    for i in range(2):
        N = data[i].stats.npts
        m = (data[i].data[-1] - data[i].data[0]) / (data[i].stats.delta * N)
        data[i].data = data[i].data - np.arange(0, N) * m
    return data

In [3]:
def __remove_nan_values(st):
    idx = []
    [idx.append(i) for i, value in enumerate(st[0].data) if not np.isnan(value)]
    st[0].data  = np.array([st[0].data[j] for j in idx])
    st[1].data  = np.array([st[1].data[j] for j in idx])
    st[2].data  = np.array([st[2].data[j] for j in idx])
    
    return st

In [4]:
def __smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [5]:
def __makeplot_calibration(st, slopes=None, intercepts=None, tf=None, setT=None, derive=None, poly=None, smooth=None, coeff=None):
    
    if derive:
        x_temp = np.gradient(st[2].data, 2)
    else:
        x_temp = st[2].data     
          
    if smooth:
        x_temp = __smooth(x_temp, smooth)

    ## adjust time axis
    diff = 86400
    
    
    fig, ax = plt.subplots(3, 2, figsize=(15,10))

    plt.subplots_adjust(hspace=0.3)
    
    ax[0][0].plot(st[0].times()/diff, st[2].data, label="Temp")
    ax[0][1].plot(st[0].times()/diff, st[2].data, label="Temp")
    
    
    if tf is not None:
        if str(type(tf)) == "<class 'pandas.core.frame.DataFrame'>":
            ax[0][0].plot(tf['Seconds']/diff, tf['Temperature (°C)'])
            ax[0][1].plot(tf['Seconds']/diff, tf['Temperature (°C)'])

        if str(type(tf)) == "<class 'obspy.core.stream.Stream'>":
            ax[0][0].plot(tf[0].times()/diff, tf[0].data)
            ax[0][1].plot(tf[0].times()/diff, tf[0].data)
    
    if setT is not None:  
        
        ax[0][0].plot(setT['Seconds']/diff, setT['Temperature'])

        
    if derive:
        axes1 = ax[0][0].twinx()
        axes1.plot(st[0].times()/diff, x_temp, 'g', label="dT/dt")
    
        axes2 = ax[0][1].twinx()
        axes2.plot(st[0].times()/diff, x_temp, 'g', label="dT/dt")
    
    
    for i in range(2):
        
        ax[1][i].plot(x_temp, st[i].data, 'k.', ms=0.2)

        if not poly:        
            ax[1][i].plot(x_temp, slopes[i]*x_temp + intercepts[i], 'r', label=f"{slopes[i]:.2e}*x")
#             ax[1][i].plot(x_temp, slopes[i]*x_temp, 'r', label=f"{slopes[i]:.2e}*x")
            
            correction = slopes[i] * x_temp + intercepts[i]
#             correction = slopes[i] * x_temp
            
            ax[2][i].plot(st[i].times()/diff, st[i].data - correction, label="corrected", zorder=2)
            ax[2][i].plot(st[i].times()/diff, st[i].data, color='grey', alpha=0.5, label="observed")

        else: 
            ax[1][i].plot(x_temp, ffit[i], 'r')
            ax[2][i].plot(st[i].times()/diff, st[i].data - poly[i], label="corrected", zorder=2)
            ax[2][i].plot(st[i].times()/diff, st[i].data - coeff[i][0], color='grey', alpha=0.5, label="observed")
            
    
    if derive:
        ax[1][0].set_xlabel("Derivative of Temperature (°C)")
        ax[1][1].set_xlabel("Derivative of Temperature (°C)")
    else:
        ax[1][0].set_xlabel("Temperature (°C)")
        ax[1][1].set_xlabel("Temperature (°C)")

    ax[0][0].set_title("Component 1")
    ax[0][1].set_title("Component 2")             

    ax[0][0].set_xlabel("Time (days)")
    ax[0][1].set_xlabel("Time (days)")
    
    ax[2][0].set_xlabel("Time (days)")
    ax[2][1].set_xlabel("Time (days)")
    
    ax[0][0].set_ylabel("Temperature (°C)")
    
    ax[1][0].set_ylabel("Tilt (rad)")
    ax[2][0].set_ylabel("Tilt (rad)")

    
    ax[0][1].legend()
    ax[0][0].legend()
    ax[1][0].legend()
    ax[1][1].legend()
    ax[2][0].legend()
   
    if derive:
        axes1.legend()
        axes2.legend()
        axes2.set_ylabel("dT/dt ")
    
    plt.plot();
    return fig

In [6]:
def __calculate_polynomial_fit(x_array, y_array, order_of_fit=2, gradient=False, smoothing=False, cc_shift=False):
    '''
    __calculate_polynomial_fit(x_array, y_array, order_of_fit=2, gradient=False, smoothing=False, cc_shift=False)
    
    INPUT: 
        - x_array:       data for x-axis as numpy array
        - y_array:       data for y-axis as numpy array
        - order_of_fit:  highest order of polynom
        - gradient:      bool
        - smoothing:     value of smoothing+
        - cc_shift:      cross-correlation lag 
    OUTPUT:
        - output directory
    '''
    
    import numpy.polynomial.polynomial as poly
    from numpy import gradient
    
    output = {}
    output['gradient'] = gradient
    output['order_of_fit'] = order_of_fit
    output['smoothing'] = smoothing
    
    ## apply time shift according to crosscorrelation lag
    if cc_shift:
        y_array = roll(y_array, -cc_shift)
    
    ## calcuate gradient, if selected
    if gradient:
        x_array = gradient(x_array, 2)
        
    ## smoothing gradient
    if smoothing:
        x_array = __smooth(x_array,  smoothing)

    ## get poly coefficients
    output['coefficients'] = poly.polyfit(x_array, y_array, order_of_fit)

    ## calulate polynomial function
    output['fit'] = poly.polyval(x_array, output['coefficients'])

    return output

In [7]:
def __calculate_linear_regression(x_array, y_array, odr_mode=None, odr_std=[None, None], derive=None, smoothing=None, cc_shift=None):
    '''
    __calculate_linear_regression(x_array, y_array, odr=None, odr_std=[None, None] derive=None, smooth=None)
  
    INPUT: 
        - x_array:       data for x-axis as numpy array
        - y_array:       data for y-axis as numpy array
        - gradient:      bool
        - smoothing:     value of smoothing+
        - odr_mode:      select orthogonal distance regression (ODR) instead of linear regression
        - odr_std:       standard deviations for x-axis data and y-axis data [std_x, std_y]
        - cc_shift:      cross-correlation lag 
    OUTPUT:
        - output:        dictionary with output values and information
    '''
    
    from scipy import odr
    from numpy import gradient, roll, power, ones, convolve
    
    def __smooth(y, box_pts):
        box = ones(box_pts)/box_pts
        y_smooth = convolve(y, box, mode='same')
        return y_smooth
    
    def f(B, x):
        '''Linear function y = m*x + b'''
        return B[0]*x + B[1]

    ## avoid error change to dummy
    if odr_std[0] is None and odr_std[1] is None:
        odr_std = [1,1]
        
    ## define output dictionary
    output = {}
    output['gradient']  = gradient
    output['smoothing'] = smoothing
    output['cc_shift']  = cc_shift
    
    ## apply time shift according to crosscorrelation lag
    if cc_shift is not None:
        y_array = roll(y_array, -cc_shift)
    
#     ## calcuate gradient, if selected
    if derive is not None:
        x_array = gradient(x_array, 2)
        
#     ## smoothing gradient
    if smoothing is not None:
        x_array = __smooth(x_array,  smoothing)

    if odr_mode is not None: 
        sx, sy = odr_std[0], odr_std[1]
        linear = odr.Model(f)
        mydata = odr.Data(x_array, y_array, wd=1./power(sx,2), we=1./power(sy,2))
        myodr = odr.ODR(mydata, linear, beta0=[1e-8, 1e-8])
        myoutput = myodr.run()
        
        output['slope'] = myoutput.beta[0]
        output['intercept'] = myoutput.beta[1]
        
    else:
        linreg = scipy.stats.linregress(x_array, y_array)
        output['slope'] = linreg.slope
        output['intercept'] = linreg.intercept

    return output

In [8]:
# def __estimate_linear_trend(st, derive=None, smooth=None):
    
#     temp = st[2].data
    
#     if derive:
#         temp = np.gradient(temp, 2)
        
#     if smooth:
#         temp = __smooth(temp, smooth)

#     ## compute linear regression of data
#     out1 = scipy.stats.linregress(temp, st[0].data)
#     out2 = scipy.stats.linregress(temp, st[1].data)
    
#     ## prepare output dictionary
#     output = {}
#     output['slope'] = [out1.slope, out2.slope]
#     output['intercept'] = [out1.intercept, out2.intercept]
    
#     return output

In [9]:
# def __get_odr_estimate(st, std=[1, 1], derive=None, smooth=None):

#     from scipy import odr

#     def f(B, x):
#         '''Linear function y = m*x + b'''
#         return B[0]*x + B[1]
    
#     output = {}
#     output['slope'] = []
#     output['intercept'] = []
    
#     for i in range(2):
        
#         y = st[i].data
#         x = st[2].data

#         if derive: 
#             x = np.gradient(x, 2)
        
#         if smooth:
#             x = __smooth(x, smooth)
        
#         sx = std[0]
#         sy = std[1]

#         linear = odr.Model(f)

#         mydata = odr.Data(x, y, wd=1./np.power(sx,2), we=1./np.power(sy,2))

# #         myodr = odr.ODR(mydata, linear, beta0=[1., 1.])
#         myodr = odr.ODR(mydata, linear, beta0=[1e-8, 1e-8])

#         myoutput = myodr.run()
        
#         output['slope'].append(myoutput.beta[0])
#         output['intercept'].append(myoutput.beta[1])
        
#     return output

### Configurations

In [10]:
## configurations
config={}

path = "/home/andbro/Desktop/tiltmeter/"

config['save'] = False

config['name'] = 'tiltmeter'

config['set_filter'] = 'n'


## tiltmeter configurations
try:
    confTilt = __readYaml('/home/brotzer/Documents/ROMY/tiltmeter/','tiltmeter_romy.conf')
    confPT = confTilt['PT']
except:
    confTilt = __readYaml('./','tiltmeter_romy.conf')
    confPT = confTilt['PT']

FileNotFoundError: [Errno 2] No such file or directory: './tiltmeter_romy.conf'

### Load Tilt Data

load data for the platform tiltmeter

In [ ]:
# name = "TestRun1_30min_intervals"
# name = "TestRun2_2hour_intervals"
name = "TestRun4_longIntervals"


st =  read(f"/home/brotzer/Documents/ROMY/tiltmeter/Temperature_Calibration/{name}/mseed/MAE.D/*")
st += read(f"/home/brotzer/Documents/ROMY/tiltmeter/Temperature_Calibration/{name}/mseed/MAN.D/*")
st += read(f"/home/brotzer/Documents/ROMY/tiltmeter/Temperature_Calibration/{name}/mseed/MAT.D/*")
st.merge()

## resampel to 1 Hz - high frequency changes don't really matter
st.resample(1.0)

## cutoff edges of tiltmeter data to avoid fast ambient adjustments
cutoff = 600 ## seconds 
st.trim(st[0].stats.starttime+cutoff, st[0].stats.endtime-cutoff)

timeFrameBeg = st[0].stats.starttime
timeFrameEnd = st[0].stats.endtime;

### Load Temperature Data

In [ ]:
path_to_files = f"/home/brotzer/Documents/ROMY/tiltmeter/Temperature_Calibration/{name}/temperature/"

filenames = ["BW.WROMY.WS9.D.2021.349","BW.WROMY.WS9.D.2021.350"]

## read data files
for i, num in enumerate(range(49,56)):
    
    filename = f'BW.WROMY.WS9.D.2021.3{num}'
    
    if i == 0:
        tf = pd.read_csv(path_to_files+filename)
    else:
        tf0 = pd.read_csv(path_to_files+filename)
        tf = pd.concat([tf, tf0], ignore_index=True)

tf.reindex()
        
## adjust Seconds vector for multiple days
tf['Seconds'] = [tf['Seconds'][i] + int(str(tf['Date'][i])[-2:])*86400 for i in range(tf['Date'].size)]
tf['Seconds'] -= int(str(tf['Date'][0])[-2:])*86400

## correct for onset of tilmeter recording
tf['Seconds'] -= __get_seconds(st[0].stats.starttime, mode='of_day')

## cut to tiltmeter recording periodf
tf = tf[tf['Seconds'] > 0]
tf.reset_index(inplace=True)

plt.scatter(tf['Seconds']/3600, tf['Temperature (°C)'], s=0.5);

In [ ]:
tr = Trace(data=np.array(tf['Temperature (°C)']))

temp = Stream(traces=tr)

temp[0].stats.network = "BW"
temp[0].stats.station = "TEMP"
temp[0].stats.channel = "T"
temp[0].stats.starttime = UTCDateTime(f"{tf['Date'][0]} {tf['Time (UTC)'][0]}")
temp[0].stats.sampling_rate = 1.0

temp = temp.trim(timeFrameBeg, timeFrameEnd)

temp

## Set Temperature

In [ ]:
def __make_set_temperature(steps, dt):
        
        temperature = np.array([])
        for i, step in enumerate(steps):
            
            if i != 0:
                slope = np.linspace(Tval, step[2], int(60/dt)) 
                temperature = np.append(temperature, slope)
            
            T = UTCDateTime(step[1])-UTCDateTime(step[0])
            data = np.ones(int(T/dt)) * step[2]
            temperature = np.append(temperature, data)
            
            Tval = step[2]
        
        totalT = UTCDateTime(steps[len(steps)-1][1]) - UTCDateTime(steps[0][0])
        taxis = np.arange(0, totalT, dt)
            
        return taxis, temperature

In [ ]:
## sampling time for calibration data
dt = 10.0 ## seconds

## set intervals for calibration data manually
steps = [["2021-12-15 14:44", "2021-12-16 08:41", 10], 
         ["2021-12-16 08:42", "2021-12-17 09:36", 20],
         ["2021-12-17 09:37", "2021-12-17 14:29", 30]]


## generate calibration temperature curve
taxis, setTemperature = __make_set_temperature(steps, dt)

## reference to starttime of tiltmeter recordings
taxis -= st[0].stats.starttime - UTCDateTime(steps[0][0])

## transfer to data frame
setT = pd.DataFrame(data=None, columns = ["Seconds", "Temperature"])
setT['Seconds'] = taxis
setT['Temperature'] = setTemperature

### Convert Data

Convert data using the specifications of the tiltmeters

In [ ]:
def __conversion(st, confPT):

    def convertTemp(trace, gain):
        Tvolt = trace.data * gain
        return confPT['calcTempCoefficients'][0] + confPT['calcTempCoefficients'][1] * Tvolt + confPT['calcTempCoefficients'][2] * Tvolt**2 + confPT['calcTempCoefficients'][3] * Tvolt**3    
        
    def convertTilt(trace, conversion, sensitivity):
        return trace.data * conversion * sensitivity
        # print( type(conversion), type(sensitivity), type(trace.data) )

    for tr in st:
        if tr.stats.channel == 'MAT':
            tr.data = convertTemp(tr, confPT['gainTemp'])
        elif tr.stats.channel == 'MAN':
            tr.data = convertTilt(tr, confPT['convPTN'], confPT['gainTilt'])
        elif tr.stats.channel == 'MAE':
            tr.data = convertTilt(tr, confPT['convPTE'], confPT['gainTilt'])

        elif tr.stats.channel == 'LAT':
            tr.data = convertTemp(tr, confBT['gainTemp'])
        elif tr.stats.channel == 'LAN':
            tr.data = convertTilt(tr, confBT['convBTN'], confBT['gainTilt'])
        elif tr.stats.channel == 'LAE':
            tr.data = convertTilt(tr, confBT['convBTE'], confBT['gainTilt'])

    print("  -> converted data")
    return st

In [ ]:
st = __conversion(st, confPT)

### Plot Data

In [ ]:
def __makeplot_data(pt, config, tf=None, setT=None):
    
    time_scale = 60
    
    font = 14
    
    
    ## _______________________
    
    fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True)
    
    for i in range(3):        
        axes[i].grid(ls=":", color="grey", zorder=0)
    
    axes[0].plot(pt[0].times()/time_scale, pt[0].data)
    axes[1].plot(pt[1].times()/time_scale, pt[1].data)
    
    
    axes[2].plot(pt[2].times()/time_scale, pt[2].data, label="internal temperature")
    if tf is not None:
        axes[2].plot(tf['Seconds']/time_scale, tf['Temperature (°C)'], label="THP sensor")
    if setT is not None:
        axes[2].plot(setT['Seconds']/time_scale, setT['Temperature'], label="calibration temperature")
    
    axes[2].legend()
    
    axes[0].set_title('Platform Tiltmeter ROMY', fontsize=font+2)
    
    axes[2].set_xlabel("Time (min)", fontsize=font)
    
    axes[0].set_ylabel("$\Omega_{east}$ (rad)", fontsize=font)
    axes[1].set_ylabel("$\Omega_{north}$ (rad)", fontsize=font)
    axes[2].set_ylabel("Temp (°C)", fontsize=font)
    
    return fig 

In [ ]:
fig = __makeplot_data(st, config, tf, setT);

## Selection 1

In [ ]:
st_selection1 = st.copy()
temp_selection1 = temp.copy()

st_selection1 = __remove_nan_values(st_selection1)


sel1_beg = UTCDateTime("2021-12-16 08:50")
sel1_end = UTCDateTime("2021-12-16 10:41")

st_selection1.trim(sel1_beg, sel1_end)

temp_selection1.trim(sel1_beg, sel1_end)

fig = __makeplot_data(st_selection1, config, tf=None, setT=None);

### Polynomial Fit

In [ ]:
x_array   = st_selection1[2].data
y_array_1 = st_selection1[0].data
y_array_2 = st_selection1[1].data

output1 = __calculate_polynomial_fit(x_array, y_array_1, order_of_fit=4, gradient=True, smoothing=100)
output2 = __calculate_polynomial_fit(x_array, y_array_2, order_of_fit=4, gradient=True, smoothing=100)

coefficients = [output1['coefficients'], output2['coefficients']]
ffit = [output1['fit'], output2['fit']]

__makeplot_calibration(st_selection1, 
                       slopes=None, 
                       intercepts=None, 
                       tf=None,
                       setT=None, 
                       derive=True, 
                       smooth=100, 
                       poly=ffit, 
                       coeff=coefficients,
                      );

## Selection 2

In [ ]:
st_selection2 = st.copy()
temp_selection2 = temp.copy()

st_selection2 = __remove_nan_values(st_selection2)

## slow adjustment at the end
sel2_beg = UTCDateTime("2021-12-18 00:00")
sel2_end = UTCDateTime("2021-12-21 00:00")

## constant plateau inbetween
# sel2_beg = UTCDateTime("2021-12-15 18:00")
# sel2_end = UTCDateTime("2021-12-16 08:00")

## increasing temperature
# sel2_beg = UTCDateTime("2021-12-15 14:00")
# sel2_end = UTCDateTime("2021-12-15 16:00")

st_selection2.trim(sel2_beg, sel2_end)

temp_selection2.trim(sel2_beg, sel2_end)

fig = __makeplot_data(st_selection2, config, tf=None, setT=None);

### Linear Regression

In [ ]:
x_array   = st_selection2[2].data
y_array_1 = st_selection2[0].data
y_array_2 = st_selection2[1].data


output1 = __calculate_linear_regression(x_array, y_array_1, odr_mode=None, odr_std=[None, None], derive=None, smoothing=None, cc_shift=None)
output2 = __calculate_linear_regression(x_array, y_array_2, odr_mode=None, odr_std=[None, None], derive=None, smoothing=None, cc_shift=None)

slopes     = [output1.get('slope'), output2.get('slope')]
intercepts = [output1.get('intercept'), output2.get('intercept')]

print(f"slopes: {slopes} \nintercepts: {intercepts}")

__makeplot_calibration(st_selection2, 
                       slopes=slopes, 
                       intercepts=intercepts, 
                       tf=None,
                       setT=None,
                       derive=False,
                       smooth=None);

## Selection 3

In [ ]:
st_selection3 = st.copy()
temp_selection3 = temp.copy()

st_selection3 = __remove_nan_values(st_selection3)


sel3_beg = UTCDateTime("2021-12-16 08:42")
sel3_end = UTCDateTime("2021-12-16 08:44")

st_selection3.trim(sel3_beg, sel3_end)

temp_selection3.trim(sel3_beg, sel3_end)

fig = __makeplot_data(st_selection3, config, tf=None, setT=None);

### Polynomial Fit

In [ ]:
x_array   = st_selection3[2].data
y_array_1 = st_selection3[0].data
y_array_2 = st_selection3[1].data

## correct for the already determined linear trend on absolute temperature
slopes     = [-5.7213704115443028e-06, 2.3228481759269141e-06] 
intercepts = [3.7118519864895699e-05, -2.7996873715913916e-05]
y_array_1 -= (slopes[0]* st_selection3[2].data + intercepts[0])
y_array_2 -= (slopes[1]* st_selection3[2].data + intercepts[1])


output1 = __calculate_polynomial_fit(x_array, y_array_1, order_of_fit=3, gradient=True, smoothing=None)
output2 = __calculate_polynomial_fit(x_array, y_array_2, order_of_fit=3, gradient=True, smoothing=None)

print(f" component 1: {output1['coefficients']} \n component 2: {output2['coefficients']}")

coefficients = [output1['coefficients'], output2['coefficients']]
ffit = [output1['fit'], output2['fit']]

__makeplot_calibration(st_selection3, 
                       slopes=None, 
                       intercepts=None, 
                       tf=None,
                       setT=None, 
                       derive=True, 
                       smooth=None, 
                       poly=ffit, 
                       coeff=coefficients,
                      );

In [ ]:
x_array   = st_selection3[2].data
y_array_1 = st_selection3[0].data
y_array_2 = st_selection3[1].data

## correct for the already determined linear trend on absolute temperature
# slopes     = [-5.7213704115443028e-06, 2.3228481759269141e-06] 
# intercepts = [3.7118519864895699e-05, -2.7996873715913916e-05]
# y_array_1 -= (slopes[0]* st_selection3[2].data + intercepts[0])
# y_array_2 -= (slopes[1]* st_selection3[2].data + intercepts[1])


output1 = __calculate_linear_regression(x_array, y_array_1, odr_mode=None, odr_std=[None, None], derive=True, smoothing=None, cc_shift=None)
output2 = __calculate_linear_regression(x_array, y_array_2, odr_mode=None, odr_std=[None, None], derive=True, smoothing=None, cc_shift=None)

slopes     = [output1.get('slope'), output2.get('slope')]
intercepts = [output1.get('intercept'), output2.get('intercept')]

print(f"slopes: {slopes} \nintercepts: {intercepts}")

__makeplot_calibration(st_selection3, 
                       slopes=slopes, 
                       intercepts=intercepts, 
                       tf=None,
                       setT=None,
                       derive=True,
                       smooth=None);

### Correct ALL

In [ ]:
def __correction(temperature, mode):
    
    import numpy.polynomial.polynomial as poly
    
    def __smooth(y, box_pts):
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
        return y_smooth
    
    dt_temperature = np.gradient(temperature, 2)/1.0 # 1 Hz
    dt_temperature = __smooth(dt_temperature, 5)
        
    slopes     = [-5.7213704115443028e-06, 2.3228481759269141e-06] 
    intercepts = [3.7118519864895699e-05, -2.7996873715913916e-05]
    
    ## section 1
    component1 = [ -1.29070103e-05,   1.17506699e+00,   7.82954151e+04,   2.04435691e+09] 
    component2 = [ -7.46312739e-06,  -2.51982477e-01,  -1.10367487e+03,   1.30426291e+08]
    ## section 3
    component1 = [ -1.37554302e-05,  -4.06760215e-04,  -1.11930295e-01,   1.11625920e+02] 
    component2 = [ -7.41299125e-06,   4.91276111e-03,  -4.73974487e+00,   1.37157381e+03]  
    
    polyfit1 = poly.polyval(dt_temperature, component1)
    polyfit2 = poly.polyval(dt_temperature, component2)
    
    if mode == 'lin':
        correction1 = slopes[0]* temperature + intercepts[0]
        correction2 = slopes[1]* temperature + intercepts[1]
    if mode == 'lin+poly':
        correction1 = slopes[0]* temperature + intercepts[0] + polyfit1
        correction2 = slopes[1]* temperature + intercepts[1] + polyfit2
    
    ## test linear dt fit
    dt_slopes     = [-0.00017277688239036794, -0.0003238346385749619] 
    dt_intercepts = [-1.3977405041720336e-05, -5.5667712989062741e-06]
    
    if mode == 'lin+lin':
        correction1 = slopes[0]* temperature + intercepts[0] + dt_slopes[0] * dt_temperature + dt_intercepts[0]
        correction2 = slopes[1]* temperature + intercepts[1] + dt_slopes[0] * dt_temperature + dt_intercepts[0] 

    return correction1, correction2, dt_temperature

In [ ]:
def __makeplot_correction(st, corrections, config, tf=None, setT=None, dtT=None):
    
    time_scale = 60
    
    font = 14
    
    
    ## _______________________
    
    fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True)
    
    for i in range(3):        
        axes[i].grid(ls=":", color="grey", zorder=0)
    
    ## panel 0
    axes[0].plot(st[0].times()/time_scale, st[0].data, label="original")
    axes[0].plot(st[0].times()/time_scale, st[0].data - corrections[0], color="green", label="corrected")

    ## panel 1
    axes[1].plot(st[1].times()/time_scale, st[1].data, label="original")
    axes[1].plot(st[1].times()/time_scale, st[1].data - corrections[1], color="green", label="corrected")
    
    ## panel 2
    axes[2].plot(st[2].times()/time_scale, st[2].data, label="internal temperature")
    
    
    if tf is not None:
        axes[2].plot(tf['Seconds']/time_scale, tf['Temperature (°C)'], label="THP sensor")
    if setT is not None:
        axes[2].plot(setT['Seconds']/time_scale, setT['Temperature'], label="calibration temperature")
    if dtT is not None:
        axes[2].plot(st[1].times()/time_scale, dtT, label="dt temperature")
        
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    
    axes[0].set_title('Platform Tiltmeter ROMY', fontsize=font+2)
    
    axes[2].set_xlabel("Time (min)", fontsize=font)
    
    axes[0].set_ylabel("$\Omega_{east}$ (rad)", fontsize=font)
    axes[1].set_ylabel("$\Omega_{north}$ (rad)", fontsize=font)
    axes[2].set_ylabel("Temp (°C)", fontsize=font)
    
    
    axes[0].set_ylim(-3e-5, 10e-5)
    axes[1].set_ylim(-8e-5, 8e-5)
    
    return fig 

In [ ]:
corr1, corr2, dtT = __correction(st[2].data, mode='lin+poly')

timeaxis = st[0].times()


__makeplot_correction(st, [corr1, corr2], config, tf=None, setT=None, dtT=dtT*10000);

## Dump Correction Parameters

In [ ]:
# import yaml

# correction = {'Tiltmeter': config.get("tiltmeter"),
#               'correction': 'LongTermTrendTemperature',
#               'slopes':  {'C1': str(out.get('slope')[0]), 'C2': str(out.get('slope')[1])},
#               'intercepts':  {'C1': str(out.get('intercept')[0]), 'C2': str(out.get('intercept')[1])},
#              }

# filename = f"correction_longterm_{config.get('tiltmeter')}"

# if config['save']:
#     with open(f'{filename}.yml', 'w') as yaml_file:
#         yaml.dump(correction, yaml_file, default_flow_style=False)

## CrossCorrelation

In [ ]:
# def __cross_correlation(y1, y2, norm=False, plot=True):
#     """Calculates the cross correlation and lags.

#     Args:
#         y1, y2: Should have the same length.
#         norm:   Boolean (False | True) for normalization 
#     Returns:
#         max_corr: Maximum correlation without normalization.
#         lag: The lag in terms of the index.
#     """
    
#     from scipy.signal import correlate
    
#     if len(y1) != len(y2):
#         raise ValueError('The lengths of the inputs should be the same.')

#     ## calulate autocorrelation
#     y1_auto_corr = np.dot(y1, y1) / len(y1)
#     y2_auto_corr = np.dot(y2, y2) / len(y1)
    
#     ## normalize input traces ?!
# #     y1 /= np.max(abs(y1))
# #     y2 /= np.max(abs(y2))
    
#     ## calculate crorrelation function
#     corr = correlate(y1, y2, mode='same')
    
#     # The unbiased sample size is N - lag.
#     unbiased_sample_size = signal.correlate(np.ones(len(y1)), np.ones(len(y1)), mode='same')

#     ## 
#     corr = corr / unbiased_sample_size / np.sqrt(y1_auto_corr * y2_auto_corr)
    
#     ## normalize
#     if norm:
#         corr /= np.max(abs(corr))
        
#     ## calculate lag times 
#     xlags = np.arange(-corr.size//2, corr.size//2, 1)

#     ## get maximum    
# #     corr_max_y = np.max(abs(corr))
#     corr_max_x = np.argmax(abs(corr))-corr.size//2
#     corr_max_y = corr[int(corr_max_x + corr.size//2)]
    
    
#     shifted = np.roll(y2*corr_max_y, corr_max_x)

    
#     ## Plotting
#     if plot == True:
#         fig, ax = plt.subplots(3, 1, figsize=(15,8))

#         ax[0].plot(y1)

#         ax[1].plot(y2)
#         ax[1].plot(shifted)

#         ax[2].plot(xlags, corr)

#         ax[2].scatter(corr_max_x, corr_max_y, color='orange', zorder=3)
#         ax[2].axvline(corr_max_x, color='k', ls=":")
#         ax[2].axhline(corr_max_y, color='k', ls=":")

#         for i in range(3):
#             ax[i].grid(ls=":", zorder=1)
            
#         plt.show();

#     return corr, xlags, (corr_max_x, corr_max_y)

In [ ]:
from andbro__crosscorrelation import __crosscorrelation

In [11]:
st_new = st.copy()

st_new.detrend('constant')

# cc, cc_lags, cc_max = __cross_correlation(st_new[2].data, st_new[1].data , norm=True, plot=True)
cc, cc_lags, cc_max = __crosscorrelation(st_new[2].data, st_new[1].data, plot=True)

NameError: name 'st' is not defined

## Selection 5

In [ ]:
st_selection5 = st.copy()
temp_selection5 = temp.copy()

st_selection5 = __remove_nan_values(st_selection5)


In [ ]:
x_array   = st_selection5[2].data
y_array_1 = st_selection5[0].data
y_array_2 = st_selection5[1].data


y_array_1 = np.roll(y_array_1, -9617)
y_array_2 = np.roll(y_array_2, -9617)

# st_selection5[0].data = y_array_1
# st_selection5[1].data = y_array_2

output1 = __calculate_linear_regression(x_array, y_array_1, odr_mode=None, odr_std=[None, None], derive=False, smoothing=None, cc_shift=None)
output2 = __calculate_linear_regression(x_array, y_array_2, odr_mode=None, odr_std=[None, None], derive=False, smoothing=None, cc_shift=None)

slopes     = [output1.get('slope'), output2.get('slope')]
intercepts = [output1.get('intercept'), output2.get('intercept')]

print(f"slopes: {slopes} \nintercepts: {intercepts}")

__makeplot_calibration(st_selection5, 
                       slopes=slopes, 
                       intercepts=intercepts, 
                       tf=None,
                       setT=None,
                       derive=None,
                       smooth=None);